# 다단계 프롬프트를 사용한 단위 테스트 작성(이전 API 사용)

단위 테스트 작성과 같은 복잡한 작업은 다중 단계 프롬프트의 이점을 활용할 수 있습니다. 단일 프롬프트와 달리 다중 단계 프롬프트는 GPT-3에서 텍스트를 생성한 다음 해당 텍스트를 후속 프롬프트에 다시 공급합니다. 이는 답변하기 전에 GPT-3가 그 이유를 설명하거나 계획을 실행하기 전에 브레인스토밍을 하려는 경우에 유용할 수 있습니다.

이 노트북에서는 3단계 프롬프트를 사용하여 다음 단계를 따라 Python으로 단위 테스트를 작성합니다:

1. Python 함수가 주어지면 먼저 GPT-3에 함수가 수행하는 작업을 설명하라는 메시지를 표시합니다.
2. 둘째, GPT-3에 해당 함수에 대한 단위 테스트 세트를 계획하라는 메시지를 표시합니다.
    - 계획이 너무 짧으면 GPT-3에게 단위 테스트에 대한 아이디어를 더 추가해달라고 요청합니다.
3. 마지막으로, GPT-3에게 단위 테스트를 작성하라는 메시지를 표시합니다.

코드 예제에서는 연쇄적인 다단계 프롬프트에 몇 가지 선택적 장식을 추가하는 방법을 설명합니다:

- 조건부 분기(예: 첫 번째 계획이 너무 짧을 경우에만 정교화 요청)
- 단계별로 다른 모델(예: 텍스트 계획 단계의 경우 `text-davinci-002`, 코드 작성 단계의 경우 `code-davinci-002`)
- 출력이 만족스럽지 않은 경우 함수를 다시 실행하는 검사(예: Python의 `ast` 모듈로 출력 코드를 파싱할 수 없는 경우)
- 출력이 완전히 생성되기 전에 출력을 읽기 시작할 수 있도록 출력 스트리밍(긴 다단계 출력에 유용)

전체 3단계 프롬프트는 다음과 같습니다(단위 테스트 프레임워크의 경우 `pytest`, 함수의 경우 `is_palindrome`을 예로 사용):

    # pytest로 훌륭한 단위 테스트를 작성하는 방법

    이 전문가용 고급 튜토리얼에서는 Python 3.9와 `pytest`를 사용하여 다음 함수의 동작을 검증하는 일련의 단위 테스트를 작성해 보겠습니다.
    ```python
    def is_palindrome(s):
        return s == s[::-1]
    ```

    단위 테스트를 작성하기 전에 함수의 각 요소가 정확히 무엇을 하고 있는지, 작성자의 의도가 무엇인지 살펴봅시다.
    - 먼저, {1단계에서 생성됨}
        
    좋은 단위 테스트 스위트의 목표는 다음과 같아야 합니다:
    - 가능한 다양한 입력에 대해 함수의 동작을 테스트합니다.
    - 작성자가 예상하지 못한 에지 케이스를 테스트한다.
    - 테스트를 쉽게 작성하고 유지 관리할 수 있도록 'pytest'의 기능을 활용합니다.
    - 깔끔한 코드와 설명적인 이름으로 읽고 이해하기 쉬워야 합니다.
    - 테스트가 항상 같은 방식으로 통과하거나 실패하도록 결정론적이어야 합니다.

    파이테스트`에는 단위 테스트를 쉽게 작성하고 유지 관리할 수 있는 편리한 기능이 많이 있습니다. 위의 함수에 대한 단위 테스트를 작성하는 데 사용하겠습니다.

    이 특정 함수의 경우 단위 테스트가 다음과 같은 다양한 시나리오를 처리하도록 할 것입니다(각 시나리오 아래에 몇 가지 예제를 하위 글머리 기호로 포함시켰습니다):
    -{2단계에서 생성됨}

    [선택적으로 추가됨]위의 시나리오 외에도 드물거나 예상치 못한 에지 케이스를 테스트하는 것을 잊지 않도록 해야 합니다(각 에지 케이스 아래에 하위 불릿으로 몇 가지 예제를 포함시켰습니다):
    -{2b단계에서 생성됨}

    개별 테스트에 들어가기 전에 먼저 단위 테스트의 전체 집합을 일관된 전체로 살펴봅시다. 각 줄이 무엇을 하는지 설명하기 위해 유용한 주석을 추가했습니다.
    ```python
    import pytest # 단위 테스트에 사용됨

    def is_palindrome(s):
        반환 s == s[::-1]

    #아래에서 각 테스트 케이스는 @pytest.mark.parametrize 데코레이터에 전달된 튜플로 표현됩니다.
    {3단계에서 생성됨}

In [1]:
# imports needed to run the code in this notebook
import ast  # used for detecting whether generated Python code is valid
import openai  # used for calling the OpenAI API

# example of a function that uses a multi-step prompt to write unit tests
def unit_test_from_function(
    function_to_test: str,  # Python function to test, as a string
    unit_test_package: str = "pytest",  # unit testing package; use the name as it appears in the import statement
    approx_min_cases_to_cover: int = 7,  # minimum number of test case categories to cover (approximate)
    print_text: bool = False,  # optionally prints text; helpful for understanding the function & debugging
    text_model: str = "text-davinci-002",  # model used to generate text plans in steps 1, 2, and 2b
    code_model: str = "code-davinci-002",  # if you don't have access to code models, you can use text models here instead
    max_tokens: int = 1000,  # can set this high, as generations should be stopped earlier by stop sequences
    temperature: float = 0.4,  # temperature = 0 can sometimes get stuck in repetitive loops, so we use 0.4
    reruns_if_fail: int = 1,  # if the output code cannot be parsed, this will re-run the function up to N times
) -> str:
    """Outputs a unit test for a given Python function, using a 3-step GPT-3 prompt."""

    # Step 1: Generate an explanation of the function

    # create a markdown-formatted prompt that asks GPT-3 to complete an explanation of the function, formatted as a bullet list
    prompt_to_explain_the_function = f"""# How to write great unit tests with {unit_test_package}

In this advanced tutorial for experts, we'll use Python 3.9 and `{unit_test_package}` to write a suite of unit tests to verify the behavior of the following function.
```python
{function_to_test}
```

Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
- First,"""
    if print_text:
        text_color_prefix = "\033[30m"  # black; if you read against a dark background \033[97m is white
        print(text_color_prefix + prompt_to_explain_the_function, end="")  # end='' prevents a newline from being printed

    # send the prompt to the API, using \n\n as a stop sequence to stop at the end of the bullet list
    explanation_response = openai.Completion.create(
        model=text_model,
        prompt=prompt_to_explain_the_function,
        stop=["\n\n", "\n\t\n", "\n    \n"],
        max_tokens=max_tokens,
        temperature=temperature,
        stream=True,
    )
    explanation_completion = ""
    if print_text:
        completion_color_prefix = "\033[92m"  # green
        print(completion_color_prefix, end="")
    for event in explanation_response:
        event_text = event["choices"][0]["text"]
        explanation_completion += event_text
        if print_text:
            print(event_text, end="")

    # Step 2: Generate a plan to write a unit test

    # create a markdown-formatted prompt that asks GPT-3 to complete a plan for writing unit tests, formatted as a bullet list
    prompt_to_explain_a_plan = f"""
    
A good unit test suite should aim to:
- Test the function's behavior for a wide range of possible inputs
- Test edge cases that the author may not have foreseen
- Take advantage of the features of `{unit_test_package}` to make the tests easy to write and maintain
- Be easy to read and understand, with clean code and descriptive names
- Be deterministic, so that the tests always pass or fail in the same way

`{unit_test_package}` has many convenient features that make it easy to write and maintain unit tests. We'll use them to write unit tests for the function above.

For this particular function, we'll want our unit tests to handle the following diverse scenarios (and under each scenario, we include a few examples as sub-bullets):
-"""
    if print_text:
        print(text_color_prefix + prompt_to_explain_a_plan, end="")

    # append this planning prompt to the results from step 1
    prior_text = prompt_to_explain_the_function + explanation_completion
    full_plan_prompt = prior_text + prompt_to_explain_a_plan

    # send the prompt to the API, using \n\n as a stop sequence to stop at the end of the bullet list
    plan_response = openai.Completion.create(
        model=text_model,
        prompt=full_plan_prompt,
        stop=["\n\n", "\n\t\n", "\n    \n"],
        max_tokens=max_tokens,
        temperature=temperature,
        stream=True,
    )
    plan_completion = ""
    if print_text:
        print(completion_color_prefix, end="")
    for event in plan_response:
        event_text = event["choices"][0]["text"]
        plan_completion += event_text
        if print_text:
            print(event_text, end="")

    # Step 2b: If the plan is short, ask GPT-3 to elaborate further
    # this counts top-level bullets (e.g., categories), but not sub-bullets (e.g., test cases)
    elaboration_needed = plan_completion.count("\n-") +1 < approx_min_cases_to_cover  # adds 1 because the first bullet is not counted
    if elaboration_needed:
        prompt_to_elaborate_on_the_plan = f"""

In addition to the scenarios above, we'll also want to make sure we don't forget to test rare or unexpected edge cases (and under each edge case, we include a few examples as sub-bullets):
-"""
        if print_text:
            print(text_color_prefix + prompt_to_elaborate_on_the_plan, end="")

        # append this elaboration prompt to the results from step 2
        prior_text = full_plan_prompt + plan_completion
        full_elaboration_prompt = prior_text + prompt_to_elaborate_on_the_plan

        # send the prompt to the API, using \n\n as a stop sequence to stop at the end of the bullet list
        elaboration_response = openai.Completion.create(
            model=text_model,
            prompt=full_elaboration_prompt,
            stop=["\n\n", "\n\t\n", "\n    \n"],
            max_tokens=max_tokens,
            temperature=temperature,
            stream=True,
        )
        elaboration_completion = ""
        if print_text:
            print(completion_color_prefix, end="")
        for event in elaboration_response:
            event_text = event["choices"][0]["text"]
            elaboration_completion += event_text
            if print_text:
                print(event_text, end="")

    # Step 3: Generate the unit test

    # create a markdown-formatted prompt that asks GPT-3 to complete a unit test
    starter_comment = ""
    if unit_test_package == "pytest":
        starter_comment = "Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator"
    prompt_to_generate_the_unit_test = f"""

Before going into the individual tests, let's first look at the complete suite of unit tests as a cohesive whole. We've added helpful comments to explain what each line does.
```python
import {unit_test_package}  # used for our unit tests

{function_to_test}

#{starter_comment}"""
    if print_text:
        print(text_color_prefix + prompt_to_generate_the_unit_test, end="")

    # append this unit test prompt to the results from step 3
    if elaboration_needed:
        prior_text = full_elaboration_prompt + elaboration_completion
    else:
        prior_text = full_plan_prompt + plan_completion
    full_unit_test_prompt = prior_text + prompt_to_generate_the_unit_test

    # send the prompt to the API, using ``` as a stop sequence to stop at the end of the code block
    unit_test_response = openai.Completion.create(
        model=code_model,
        prompt=full_unit_test_prompt,
        stop="```",
        max_tokens=max_tokens,
        temperature=temperature,
        stream=True
    )
    unit_test_completion = ""
    if print_text:
        print(completion_color_prefix, end="")
    for event in unit_test_response:
        event_text = event["choices"][0]["text"]
        unit_test_completion += event_text
        if print_text:
            print(event_text, end="")

    # check the output for errors
    code_start_index = prompt_to_generate_the_unit_test.find("```python\n") + len("```python\n")
    code_output = prompt_to_generate_the_unit_test[code_start_index:] + unit_test_completion
    try:
        ast.parse(code_output)
    except SyntaxError as e:
        print(f"Syntax error in generated code: {e}")
        if reruns_if_fail > 0:
            print("Rerunning...")
            return unit_test_from_function(
                function_to_test=function_to_test,
                unit_test_package=unit_test_package,
                approx_min_cases_to_cover=approx_min_cases_to_cover,
                print_text=print_text,
                text_model=text_model,
                code_model=code_model,
                max_tokens=max_tokens,
                temperature=temperature,
                reruns_if_fail=reruns_if_fail-1,  # decrement rerun counter when calling again
            )

    # return the unit test as a string
    return unit_test_completion


In [2]:
example_function = """def is_palindrome(s):
    return s == s[::-1]"""

unit_test_from_function(example_function, print_text=True)

# How to write great unit tests with pytest

In this advanced tutorial for experts, we'll use Python 3.9 and `pytest` to write a suite of unit tests to verify the behavior of the following function.
```python
def is_palindrome(s):
    return s == s[::-1]
```

Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
- First, we have a function definition. This is where we give the function a name, `is_palindrome`, and specify the arguments that the function accepts. In this case, the function accepts a single string argument, `s`.
- Next, we have a return statement. This is where we specify the value that the function returns. In this case, the function returns `s == s[::-1]`.
- Finally, we have a function call. This is where we actually call the function with a specific set of arguments. In this case, we're calling the function with the string `"racecar"`.
    
A good unit test suite should aim to:
- 

'.\n#The first element of the tuple is a name for the test case, and the second element is a list of arguments for the test case.\n#The @pytest.mark.parametrize decorator will generate a separate test function for each test case.\n#The generated test function will be named test_is_palindrome_<name> where <name> is the name of the test case.\n#The generated test function will be given the arguments specified in the list of arguments for the test case.\n#The generated test function will be given the fixture specified in the decorator, in this case the function itself.\n#The generated test function will call the function with the arguments and assert that the result is equal to the expected value.\n@pytest.mark.parametrize(\n    "name,args,expected",\n    [\n        # Test the function\'s behavior for a wide range of possible inputs\n        ("palindrome", ["racecar"], True),\n        ("palindrome", ["madam"], True),\n        ("palindrome", ["anna"], True),\n        ("non-palindrome", ["p